In [1]:

#!pip3 install tensorflow==2.0.0-beta1
#!pip3 install tensorflow_datasets
%matplotlib inline
%load_ext tensorboard
import os
import datetime
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras as tfk
import numpy as np
import tensorflow.keras.layers as tfkl
import tensorflow_datasets as tfds

tfds.disable_progress_bar()
print(tf.__version__)
log_dir = "logd"
!rm -rf logd/

2.0.0-beta1


In [2]:
def plot_filters(model,ii):
    d = model.layers[0]
    o = d(ii)
    mainImg=[]
    for i in range(8): 
        a = []
        for j in range(8):
            a.append(o[i,:,:,j])
        imgg = np.hstack(a)
        mainImg.append(imgg)
    return np.vstack(mainImg)

In [3]:
class Model(tfk.Model):
  def __init__(self):
    super(Model,self).__init__()
    self.common_layers=[]
    self.common_layers.append(tfkl.Conv2D(16,3,strides=(1,1),padding='same',activation="relu",input_shape=(28,28,1)))
    #self.common_layers.append(tfkl.Conv2D(64,3,strides=(1,1),padding='same',activation="relu"))
    #self.common_layers.append(tfkl.MaxPool2D((2,2),(2,2)))
    self.common_layers.append(tfkl.Flatten())
    #self.common_layers.append(tfkl.Dense(128,activation="relu"))
    self.common_layers.append(tfkl.Dense(32,activation="relu"))
    self.common_layers.append(tfkl.Dense(10,activation="softmax"))
                              
  def call(self,x):
    #x= tf.convert_to_tensor(x,dtype=tf.float32)
    for layer in self.common_layers:
      x = layer(x)
    return x


In [4]:
model = Model()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logd/gradient_tape/' + current_time + '/train'

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [6]:
ii = x_train[:10,:,:,:]

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(0.001)
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


In [8]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)


In [9]:
train_summary_writer = tf.summary.create_file_writer(train_log_dir)


In [10]:
EPOCHS = 8
#tf.summary.trace_on(graph=True, profiler=True)
for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
        filter_img = plot_filters(model,ii)
        filter_img = tf.summary.image('image',filter_img[np.newaxis,:,:,np.newaxis],step=epoch)
        #tf.summary.trace_export(name="my_func_trace", step=0, profiler_outdir=train_log_dir)

    for test_images, test_labels in test_ds:
      test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch 1, Loss: 0.18967749178409576, Accuracy: 94.37833404541016, Test Loss: 0.09249943494796753, Test Accuracy: 97.11000061035156
Epoch 2, Loss: 0.06901370733976364, Accuracy: 97.94833374023438, Test Loss: 0.08133405447006226, Test Accuracy: 97.44499969482422
Epoch 3, Loss: 0.04256151616573334, Accuracy: 98.76333618164062, Test Loss: 0.07690677046775818, Test Accuracy: 97.59333038330078
